# Stock Price Prediction: ARIMA vs LSTM

This notebook walks through the process of predicting stock prices using two different approaches: a traditional statistical model (ARIMA) and a deep learning model (LSTM). I'll be using simulated data that mimics historical stock prices to compare how these models handle trends and fluctuations.

## 1. Setting Up and Getting Data
First, I'll import the necessary libraries and generate some sample data. In a real-world scenario, I'd pull this from Yahoo Finance, but for this exercise, I'm creating a synthetic dataset with a clear trend and some noise.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Generating synthetic stock data
np.random.seed(42)
dates = pd.date_range(start='2019-01-01', end='2023-12-31', freq='B')
n = len(dates)
price = 150 + np.cumsum(np.random.normal(0.1, 1.5, n)) + 10 * np.sin(np.linspace(0, 10, n))
df = pd.DataFrame({'Date': dates, 'Adj Close': price})
df.set_index('Date', inplace=True)

plt.figure(figsize=(12,6))
plt.plot(df['Adj Close'])
plt.title('Historical Stock Price (Simulated)')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.show()

## 2. Preprocessing
Before feeding data into the models, I need to split it into training and testing sets. For the LSTM, I also need to normalize the values between 0 and 1 because neural networks are sensitive to the scale of input data.

In [ ]:
# Splitting 80/20
train_size = int(len(df) * 0.8)
train_data, test_data = df.iloc[:train_size], df.iloc[train_size:]

# Scaling for LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(train_data)
scaled_test = scaler.transform(test_data)

print(f"Training samples: {len(train_data)}")
print(f"Testing samples: {len(test_data)}")

## 3. ARIMA Model
ARIMA is a classic for time series. I'll use a simple (5,1,0) configuration here. I'm using a rolling forecast approach where the model gets the actual previous value to predict the next one, which is a common way to evaluate these models.

In [ ]:
# Quick look at ACF/PACF to justify parameters
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,4))
plot_acf(df['Adj Close'].diff().dropna(), ax=ax1)
plot_pacf(df['Adj Close'].diff().dropna(), ax=ax2)
plt.show()

# Running the ARIMA forecast
history = [x for x in train_data['Adj Close']]
predictions_arima = []
for t in range(len(test_data)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit()
    yhat = model_fit.forecast()[0]
    predictions_arima.append(yhat)
    history.append(test_data.iloc[t]['Adj Close'])

plt.figure(figsize=(12,6))
plt.plot(test_data.index, test_data['Adj Close'], label='Actual')
plt.plot(test_data.index, predictions_arima, color='red', label='ARIMA Prediction')
plt.legend()
plt.title('ARIMA Forecast vs Actual')
plt.show()

## 4. LSTM Model
Now for the deep learning part. I'm setting up a sliding window of 60 days to predict the 61st. The model has a couple of LSTM layers with Dropout to prevent it from just memorizing the training data.

In [ ]:
def create_dataset(dataset, look_back=60):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 60
full_scaled = np.concatenate((scaled_train, scaled_test), axis=0)
X_train, y_train = create_dataset(scaled_train, look_back)
X_test, y_test = create_dataset(full_scaled[train_size - look_back:], look_back)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Building the model
model_lstm = Sequential([
    LSTM(50, return_sequences=True, input_shape=(look_back, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

model_lstm.compile(optimizer='adam', loss='mean_squared_error')
# Training (keeping epochs low for the demo)
history_lstm = model_lstm.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1, verbose=1)

# Predictions
predictions_lstm = model_lstm.predict(X_test)
predictions_lstm = scaler.inverse_transform(predictions_lstm)

## 5. Comparison and Results
Finally, let's see how they stack up. I'll calculate the standard error metrics and plot everything together. Usually, ARIMA is better at short-term trends while LSTM can sometimes pick up on more complex, non-linear patterns if given enough data.

In [ ]:
actual = test_data['Adj Close'].values
pred_arima = np.array(predictions_arima)
pred_lstm = predictions_lstm.flatten()

def get_metrics(a, p):
    return mean_absolute_error(a, p), np.sqrt(mean_squared_error(a, p))

mae_a, rmse_a = get_metrics(actual, pred_arima)
mae_l, rmse_l = get_metrics(actual, pred_lstm)

print(f"ARIMA -> MAE: {mae_a:.2f}, RMSE: {rmse_a:.2f}")
print(f"LSTM  -> MAE: {mae_l:.2f}, RMSE: {rmse_l:.2f}")

plt.figure(figsize=(14,7))
plt.plot(test_data.index, actual, label='Actual Price', alpha=0.7)
plt.plot(test_data.index, pred_arima, label='ARIMA', linestyle='--')
plt.plot(test_data.index, pred_lstm, label='LSTM', linestyle=':')
plt.title('Model Comparison: ARIMA vs LSTM')
plt.legend()
plt.show()